In [1]:
# library
import numpy as np
import pandas as pd
import plotly.express as px

import json

site_list = ["foxnews_science", "newsweek_science", "slate_science", "washingtontimes_science", "theatlantic_science"] 
# nothing is in cnn science

In [6]:
def cleanDF(df):
    # clean the data
    # drop the main text that is empty
    df = df.dropna(subset=["maintext"])

    # drop the duplicates
    df = df.drop_duplicates(subset=["maintext"])

    # add a new column based on the wayback_id
    df["wayback_time"] = pd.to_datetime((df["wayback_id"]).astype(str).str[:8], format='%Y%m%d', errors='coerce')

    # add a new column based on the length of the main text
    df["text_len"] = df["maintext"].apply(lambda x: len(x))

    df.drop(columns=["wayback_id"], inplace=True)

    # sort
    df = df.sort_values(by="wayback_time", ascending=True)

    # print(df.describe())
    # print(df.head())
    # df.head()
    return df

In [15]:
def open_article_json(file_path):
    with open(file_path) as f:
        json_data = json.load(f)
    data = [{"url": url, **details} for url, details in json_data.items() if details is not None]
    df = pd.DataFrame(data)
    df = df[["title", "authors", "url", "date_publish", "description", "maintext", "wayback_id"]]
    df["category"] = "Science"
    return df

In [16]:
for site in site_list:  
    file_path = f"./data/{site}/articles.json"
    df = open_article_json(file_path)
    df = cleanDF(df)
    df.to_json(f"./data/{site}/{site}_categorized.json", orient='records', lines=True)